# Fluid biomarker analysis
## From Biospecimen file
### Covariates
* ApoE Genotype (covariate)
* CSF Hemoglobin (covariate)
* SNCA_multiplication

### Outcomes
1. Cross-sectional analysis (Count > 300 for people with PD/Control)
* RNAs
* serum_IGF-1

2. Longitudinal analysis (Count > 900 for people with PD/Control)
* CSF
    * Abeta 1-42
    * pTau, tTau
    * Alpha-synuclein
* MTDNAs
* RNAs (very few follow-ups for controls.)

Check the distribution and conduct necessary transformation.



=================Start====================


## Look at the observation duplications

In [1]:
library(dplyr);library(data.table);library(ggplot2)
t = fread('../PDcohorts/PPMI/download181018/Current_Biospecimen_Analysis_Results.csv')
t[t==""]=NA
t[t=="N/A"]=NA
t[t=="Undetermined"]=NA

t = t %>% 
  arrange(desc(1:nrow(.))) %>% # arrange descending to get the last with distinct
  group_by(PATNO, TYPE, TESTNAME, CLINICAL_EVENT, PROJECTID) %>% 
  mutate(N = n()) %>% ungroup()
t %>% with(table(PROJECTID, N)) # Duplication exists



Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'data.table' was built under R version 3.5.2"
Attaching package: 'data.table'

The following objects are masked from 'package:dplyr':

    between, first, last

Warning message:
"package 'ggplot2' was built under R version 3.5.2"Warning message:
"package 'bindrcpp' was built under R version 3.5.2"

         N
PROJECTID     1     2     3
      104   694     0     0
      105  5876  1130   114
      106 21665     0     0
      112  4572     0     0
      113   596     0     0
      114     0  4000     0
      119 15921     0     0
      122  6749     0     0
      123   794   396     0
      124  2900   170     0
      125  8700   510     0
      127     1    20     0
      129    11    44     0
      130  3768     0     0
      134  2900   170     0

The table indicates that some observations were duplicated/multiple. > keep the last record.

## Cross sectional obs (PD and HC > 300)

In [2]:
t1 = t %>%
  distinct(PATNO, TYPE, TESTNAME, CLINICAL_EVENT, PROJECTID, .keep_all = T) %>% 
  filter(!grepl('^rs', TESTNAME)) %>% 
  filter(!grepl('rep\ 1', TESTNAME)) %>%
  filter(!grepl('SCORE', TESTNAME)) %>% 
  mutate(TEST = paste(TYPE, TESTNAME, sep= "_"))

longOUTCOME = t1 %>% filter(CLINICAL_EVENT != "BL") %>%
  distinct(TEST) %>% t %>% as.vector

CS = t1 %>% filter(!(TEST %in% longOUTCOME)) %>%
  filter(DIAGNOSIS %in% c("PD", "Control")) %>% 
  group_by(TEST) %>% filter(n()>300) %>%
  ungroup
CS %>% with(table(TEST, DIAGNOSIS))
csOutcomes = CS %>% distinct(TEST) %>% t %>% as.vector

                   DIAGNOSIS
TEST                Control  PD
  RNA_DHPR              139 202
  RNA_DJ-1              139 202
  RNA_FBXO7-001         139 202
  RNA_FBXO7-005         139 202
  RNA_FBXO7-007         139 202
  RNA_FBXO7-008         139 202
  RNA_FBXO7-010         139 202
  RNA_GLT25D1           139 202
  RNA_GUSB              139 202
  RNA_MON1B             139 202
  RNA_RPL13             139 202
  RNA_SNCA-007          139 202
  RNA_SNCA-3UTR-1       139 202
  RNA_SNCA-3UTR-2       139 202
  RNA_SNCA-E3E4         139 202
  RNA_SNCA-E4E6         139 202
  RNA_SRCAP             139 202
  RNA_UBC               139 202
  RNA_ZNF746            139 202
  Serum_Serum IGF-1     191 405

## Longitudinal obs (PD and HC > 900)

In [3]:
LT = t1 %>% filter(TEST %in% longOUTCOME) %>%
  filter(DIAGNOSIS %in% c("PD", "Control")) %>% 
  group_by(TEST) %>% filter(n()>900) %>%
  ungroup
LT %>% mutate(TEST = paste(TEST, DIAGNOSIS, sep= "__")) %>% 
  with(table(TEST, CLINICAL_EVENT))
ltOutcome = LT %>% distinct(TEST) %>% t %>% as.vector

# # Delete small number columns
# LT %>% mutate(TEST = paste(TEST, DIAGNOSIS, sep= "__")) %>% 
#   filter(!(CLINICAL_EVENT %in% c("UO1", "V01", "V05", "V07"))) %>%
#   with(table(TEST, CLINICAL_EVENT))


                                                  CLINICAL_EVENT
TEST                                                BL  ST U01 V01 V02 V04 V05
  Cerebrospinal Fluid_ABeta 1-42__Control          190   0   1   4 159 153   1
  Cerebrospinal Fluid_ABeta 1-42__PD               414 131   1   4 252 305   1
  Cerebrospinal Fluid_CSF Alpha-synuclein__Control 190   0   1   4 159 153   1
  Cerebrospinal Fluid_CSF Alpha-synuclein__PD      414 131   1   4 252 305   1
  Cerebrospinal Fluid_CSF Hemoglobin__Control      190   0   1   4 159 153   1
  Cerebrospinal Fluid_CSF Hemoglobin__PD           414 131   1   4 252 305   1
  Cerebrospinal Fluid_MTDNA_DELETION__Control      176   0   0   0   0   0   0
  Cerebrospinal Fluid_MTDNA_DELETION__PD           380   0   0   0   0   0   0
  Cerebrospinal Fluid_MTDNA_ND1_CN__Control        176   0   0   0   0   0   0
  Cerebrospinal Fluid_MTDNA_ND1_CN__PD             380   0   0   0   0   0   0
  Cerebrospinal Fluid_MTDNA_ND4_CN__Control        176   0   0   0

## Check the distributions of outcomes at baseline 1
Raw Data

In [4]:
W_crude = t1 %>% filter(CLINICAL_EVENT=="BL") %>% 
  select(PATNO, DIAGNOSIS, TEST, TESTVALUE) %>% 
  filter(TEST %in% c(csOutcomes, ltOutcome)) %>% 
  tidyr::spread(key = "TEST", value = "TESTVALUE")

# Need to be checked after
t = W_crude
t$`Cerebrospinal Fluid_pTau`[t$`Cerebrospinal Fluid_pTau`=="<8"] = NA 
t$`Cerebrospinal Fluid_tTau`[t$`Cerebrospinal Fluid_tTau`=="<80"] = NA 
t$`Cerebrospinal Fluid_CSF Hemoglobin`[t$`Cerebrospinal Fluid_CSF Hemoglobin`=='below'] = NA
t$`Cerebrospinal Fluid_CSF Hemoglobin`[t$`Cerebrospinal Fluid_CSF Hemoglobin`=='above'] = NA

convertNumColumns = function(df, rows_vec = c()){
  # Convert columns with numbers to numeric. (judge from the rows_vec.)
  # Default, look at the first and the last lines.
  rows_vec = union(c(1, nrow(df)), rows_vec) # at least look at first and last lines
  numcol = lapply(rows_vec, function(x){
    c(names(df)[suppressWarnings(!is.na(as.numeric(as.character(df[x,]))))],
      names(df)[is.na(df[x,])]) # NA cols can be number
  })
  numcols = Reduce(intersect, numcol)
  df %>% mutate_at(vars(numcols), as.numeric) %>% arrange(PATNO)
}
W = convertNumColumns(t, 1:nrow(t))
summary(W)

     PATNO        DIAGNOSIS         Cerebrospinal Fluid_ABeta 1-42
 Min.   : 3000   Length:931         Min.   : 208.7                
 1st Qu.: 3370   Class :character   1st Qu.: 639.5                
 Median : 3760   Mode  :character   Median : 858.7                
 Mean   :14573                      Mean   : 932.7                
 3rd Qu.:16612                      3rd Qu.:1144.0                
 Max.   :92834                      Max.   :3707.0                
                                    NA's   :34                    
 Cerebrospinal Fluid_CSF Alpha-synuclein Cerebrospinal Fluid_CSF Hemoglobin
 Min.   : 398.8                          Min.   :  20.0                    
 1st Qu.:1091.9                          1st Qu.:  43.1                    
 Median :1452.0                          Median :  98.7                    
 Mean   :1570.8                          Mean   : 189.4                    
 3rd Qu.:1892.3                          3rd Qu.: 219.8                    
 Max.   

In [5]:
par(mfrow=c(4,3))
draw_density = function(df, col, thres = 4, mcex = 1){
  t = df[, col] %>% t %>% as.vector
  miss = is.na(t)
  N = sum(!miss)
  X = t[!miss]
  M = mean(X)
  S = sd(X)
  Out = (X < M - thres*S) + (X > M + thres*S)
  Noutlr = sum(Out)
  X=X[Out==0]
  M = mean(X)
  S = sd(X)
  plot(density(X), main=sprintf("%s, N=%.0f \n (%.0f outlier excluded)", col, N-Noutlr, Noutlr), cex.main = mcex)
  curve(dnorm(x, M, S), min(X), max(X), add=T, col="green")
  t[Out==1] = NA
  return(t)
}
png('figure/raw.png', width=800, height=1800)
par(mfcol=c(9,4))
t = lapply(names(W)[-(1:2)], function(x) draw_density(W, x, mcex = 1.3))
dev.off()

png 
  2

[The density plot of raw data](figure/raw.png) are right skewed.    
Also, CSF hemoglobin and MT-DNAs have strange distributions.

Try log transformation below.     
MT-DNAs have the value of 0 -> excluded for this analysis

In [6]:
rmv = apply(W, 2, function(x) min(x, na.rm = T)) %>% as.numeric # remove the columns with minimum 0
rmv_col = which(rmv==0)
df = W[, -c(1:2, rmv_col)]
df_log = log(df)
png('figure/log.png', width=800, height=1800)
par(mfcol=c(9,4))
t = lapply(names(df), function(x) draw_density(df_log, x, mcex = 1.3)) %>% do.call(cbind, .) %>% data.frame()
names(t) = names(df)
W_log = data.frame(W[, 1:2], t)
dev.off()

Warning message in function_list[[k]](value):
"NAs introduced by coercion"

png 
  2

[The density plots of log-transformed data](figure/log.png) show the better fits except for CSF-Hemoglobin still doesn't fit.

<a href="figure/log.png" target="_blank">example</a>

In [7]:
png('figure/rawVSlog.png', width=1200, height=300)
par(mfcol=c(1,4))
t=lapply(c('Cerebrospinal Fluid_ABeta 1-42', 'Cerebrospinal Fluid_CSF Alpha-synuclein'), function(x) draw_density(W, x, mcex = 1))
t=lapply(c('Cerebrospinal Fluid_ABeta 1-42', 'Cerebrospinal Fluid_CSF Alpha-synuclein'), function(x) draw_density(df_log, x, mcex = 1))
dev.off()

png 
  2

[Examples of raw vs log](figure/rawVSlog.png)

In [8]:
png('figure/raw_problem.png', width=1200, height=300)
par(mfcol=c(1,4))
CSF_Hb = W$`Cerebrospinal Fluid_CSF Hemoglobin`
CSF_Hb[W_crude$`Cerebrospinal Fluid_CSF Hemoglobin`=='below'] = NA
CSF_Hb[W_crude$`Cerebrospinal Fluid_CSF Hemoglobin`=='above'] = 1500
hist(CSF_Hb, cex.main = 1.2, breaks = 20)
hist(W$`Cerebrospinal Fluid_MTDNA_DELETION`, cex.main = 1.2, breaks = 20)
hist(W$`Cerebrospinal Fluid_MTDNA_ND1_CN`, cex.main = 1.2, breaks = 20)
hist(W$`Cerebrospinal Fluid_MTDNA_ND4_CN`, cex.main = 1.2, breaks = 20)
dev.off()

png 
  2

[None normal distributions in raw or log (histgrams of raw data)](figure/raw_problem.png)

In [9]:
png('figure/CSF_HbEffect_plus.png', width=1200, height=300)
par(mfcol=c(1,4))
CSFmarkerIdx = setdiff(grep("Cerebrospinal", names(W_log)), grep("Hemoglobin", names(W_log)))
for (i in CSFmarkerIdx){
  print(i)
  Y = W_log[,i]
#   X = log(W$`Cerebrospinal Fluid_CSF Hemoglobin`)
#   X = log(CSF_Hb)
  X = CSF_Hb
  df = data.frame(X=X, Y=Y) %>% filter(complete.cases(.)) %>% arrange(X)
  plot(df, ylab = names(W_log)[i], xlab = 'log(CSF_Hb)')
  lw = loess(Y~X, data = df) 
  lines(df$X, lw$fitted, col = 'red')
}
dev.off()

[1] 3
[1] 4
[1] 6
[1] 7


png 
  2

#### Figures to assess the effect of CSF-Hb
CSF_Hb are labelled as 'below' or 'above' when it wasn't in the measurable range. 
1. [log(CSF-ASNA) vs raw CSF-Hb](figure/CSF_HbEffect.png)
2. [log(CSF-ASNA) vs raw CSF-Hb (giving 5 'below' and 1500 for 'above' for CSF-Hb](figure/CSF_HbEffect_plus.png)
3. [log(CSF-ASNA) vs log CSF-Hb](figure/CSF_HbEffect_log.png)
4. [log(CSF-ASNA) vs log CSF-Hb (giving 5 'below' and 1500 for 'above' for CSF-Hb](figure/CSF_HbEffect_log_plus.png)

The effect on CSF-Hb on CSF-ASYN are not apparent. 

In [12]:
require(ggplot2)
# png("figure/chronological.png", width = 1800, height = 600)
# par(mfcol = c(2,6))
p = list()
for (C in ltOutcome[-(1:4)]){ # first 4 are CSF_Hb and MTDNAs
    d = LT %>% filter(TEST == C) %>% mutate(X = log(as.numeric(TESTVALUE))) %>%
      filter(CLINICAL_EVENT %in% c("BL", "V02", "V04", "V06", "V08")) %>%
      select(PATNO, DIAGNOSIS, CLINICAL_EVENT, X) 
    p[[C]] = ggplot(data = d, aes(x = CLINICAL_EVENT, y = X)) +
      geom_boxplot(aes(fill=DIAGNOSIS)) + 
      ylab(paste(C, "(log)"))
}
dev.off()
library(gridExtra)
grid.arrange(grobs = p, ncol = 2)
ggsave("figure/chronological.png", arrangeGrob(grobs=p, ncol = 6), width = 24, height = 8, dpi=100)

Warning message in evalq(log(as.numeric(TESTVALUE)), <environment>):
"NAs introduced by coercion"Warning message in evalq(log(as.numeric(TESTVALUE)), <environment>):
"NAs introduced by coercion"Warning message in evalq(log(as.numeric(TESTVALUE)), <environment>):
"NAs introduced by coercion"

null device 
          1

Warning message:
"Removed 42 rows containing non-finite values (stat_boxplot)."Warning message:
"Removed 193 rows containing non-finite values (stat_boxplot)."Warning message:
"Removed 31 rows containing non-finite values (stat_boxplot)."Warning message:
"Removed 4 rows containing non-finite values (stat_boxplot)."Warning message:
"Removed 1 rows containing non-finite values (stat_boxplot)."Warning message:
"Removed 4 rows containing non-finite values (stat_boxplot)."Warning message:
"Removed 42 rows containing non-finite values (stat_boxplot)."Warning message:
"Removed 193 rows containing non-finite values (stat_boxplot)."Warning message:
"Removed 31 rows containing non-finite values (stat_boxplot)."Warning message:
"Removed 4 rows containing non-finite values (stat_boxplot)."Warning message:
"Removed 1 rows containing non-finite values (stat_boxplot)."Warning message:
"Removed 4 rows containing non-finite values (stat_boxplot)."

#### Figures to assess the chronological change over longitudinal outcomes
[Figures](figure/chronological.png)
1. CSF_markers are consistently lower for PD groups over time. No apparent chronological effect
2. For RNAs, BL are lower than others across different RNAs. Seems like a batch effect. Safer not to test a chronological effect unless we are sure about whether or not we need to correct these.
